In [ ]:
import pandas as pd
import numpy as np
import random
import csv
import io
from collections import defaultdict

# Configuración inicial
NUM_PACIENTES = 2000
GENERO_DISTRIBUCION = {"Femenino": 0.8, "Masculino": 0.2}
COMORBIDITY_RATE = 0.6
COMORBILIDADES = {
    'Depresión': 0.30,
    'Ansiedad': 0.20,
    'Trastorno bipolar': 0.02,
    'Insomnio': 0.30,
    'Intestino irritable': 0.30,
    'Enfermedades cardiovasculares': 0.10,
    'Epilepsia': 0.01
}
ADVERSE_EFFECTS_RATE = 0.25  # 25% de los IDs tendrán efectos adversos

# Carga de datos de efectos adversos
df_farmacos = pd.read_excel("BBDD_algoritmo_Migraitech.xlsx")
df_farmacos.dropna(subset=["Farmaco"], inplace=True)
farmacos_disponibles = df_farmacos["Farmaco"].dropna().unique().tolist()
farmaco_por_defecto = random.choice(farmacos_disponibles) if farmacos_disponibles else "Farmaco_Por_Defecto"
df_farmacos["Farmaco"] = df_farmacos["Farmaco"].fillna(farmaco_por_defecto)

# Construcción del diccionario de efectos adversos
ef_adversos_dict = {}
for _, row in df_farmacos.iterrows():
    farmaco = str(row["Farmaco"]).strip()
    if farmaco not in ef_adversos_dict:
        ef_adversos_dict[farmaco] = []
    ef_adversos_dict[farmaco].append({
        "Efecto_Adverso": row["Efecto_Adverso"],
        "Gravedad": row["Gravedad"],
        "Frecuencia": row["Frecuencia"]
    })

# Función para generar edad basada en la categoría de IMC y el género
def generar_edad(imc_categorica, genero):
    rangos_edad = [
        (18, 34),  # 18-34 años
        (35, 54),  # 35-54 años
        (55, 85),  # 55-77 años
        #(76, 85)   # 76-85 años
    ]
    probabilidades = [48, 36.2, 15.8]
    probabilidades_normalizadas = [p / sum(probabilidades) for p in probabilidades]
    rango_elegido = np.random.choice(len(rangos_edad), p=probabilidades_normalizadas)
    edad = random.randint(rangos_edad[rango_elegido][0], rangos_edad[rango_elegido][1])
    return edad

# Distribución de fármacos
FARMACO_DISTRIBUCION = {
    "Amitriptilina": 0.570,
    "Propranolol": 0.003,
    "Topiramato": 0.306,
    "Erenumab": 0.032,
    "Fremanezumab": 0.049,
    "Galcanezumab": 0.040
}

# Modificación de ADHERENCIA con distribución normal
ADHERENCIA = {
    "Amitriptilina": lambda: max(0, min(np.random.normal(0.549, 0.0255), 1)),
    "Propranolol": lambda: max(0, min(np.random.normal(0.77, 0.0198), 1)),
    "Topiramato": lambda: max(0, min(np.random.normal(0.569, 0.0136), 1)),
    "Erenumab": lambda: 1.0,
    "Fremanezumab": lambda: 1.0,
    "Galcanezumab": lambda: 1.0
}

# Modificación de EFICACIA con distribución normal
EFICACIA = {
    "Amitriptilina": lambda: max(0, np.random.normal(0.9, 14.53)),
    "Propranolol": lambda: max(0, np.random.normal(0.69, 9.79)),
    "Topiramato": lambda: max(0, np.random.normal(0.73, 11.39)),
    "Erenumab": lambda: max(0, np.random.normal(6.6, 5.99)),
    "Fremanezumab": lambda: max(0, np.random.normal(5.0, 6.95)),
    "Galcanezumab": lambda: max(0, np.random.normal(4.83, 7.06))
}

PRECIO = {"Amitriptilina": 16.96, "Propranolol": 110.88, "Topiramato": 236.5, "Erenumab": 5340, "Fremanezumab": 5976, "Galcanezumab": 5400}

# Restricciones de fármacos
RESTRICCIONES_FARMACOS = {
    "Topiramato": [
        ("género", "Femenino", 18, 38),  # Mujeres entre 18 y 38 años
        ("comorbilidad", "Lactancia"),
        ("comorbilidad", "Embarazo"),
        ("imc", 1),  # Bajo peso: IMC < 18.5
        ("imc", 2)   # Peso normal: 18.5 ≤ IMC < 25
    ],
    "Amitriptilina": [
        ("comorbilidad", "Trastorno bipolar"),
        ("comorbilidad", "Manía"),
        ("comorbilidad", "Enfermedades cardiovasculares"),
        ("comorbilidad", "Hepatopatía grave")
    ],
    "Propranolol": [
        ("comorbilidad", "Enfermedades cardiovasculares"),
        ("comorbilidad", "Acidosis metabólica"),
        ("comorbilidad", "Asma"),
        ("comorbilidad", "EPOC"),
        ("comorbilidad", "Síndrome Raynaud"),
        ("comorbilidad", "Angina hipoglicémica")
    ],
    "Erenumab": [
        ("comorbilidad", "Embarazo"),
        ("comorbilidad", "Lactancia")
    ],
    "Fremanezumab": [
        ("comorbilidad", "Embarazo"),
        ("comorbilidad", "Lactancia")
    ],
    "Galcanezumab": [
        ("comorbilidad", "Embarazo"),
        ("comorbilidad", "Lactancia")
    ]
}

# Función para verificar restricciones de fármacos
def verificar_restricciones(farmaco, genero, edad, comorbilidades, imc_categorica):
    if farmaco in RESTRICCIONES_FARMACOS:
        for restriccion in RESTRICCIONES_FARMACOS[farmaco]:
            tipo_restriccion, valor = restriccion[0], restriccion[1]
            if tipo_restriccion == "género":
                if genero == valor and restriccion[2] <= edad <= restriccion[3]:
                    return False
            elif tipo_restriccion == "comorbilidad":
                if f"Comorbilidad_{valor}" in comorbilidades and comorbilidades[f"Comorbilidad_{valor}"] == 1:
                    return False
            elif tipo_restriccion == "imc":
                if imc_categorica == valor:
                    return False
    return True

# Semilla para reproducibilidad
random.seed(123456)
np.random.seed(123456)

# Generación de datos
output = io.StringIO()
csv_writer = csv.writer(output)

# Crear columnas para comorbilidades y efectos adversos
columnas = [
    "id", "edad", "peso", "altura", "IMC", "IMC_categorica", "genero", "farmaco", "Adherencia", "Precio", "Eficacia"
]

# Añadir columnas para comorbilidades
for comorbilidad in COMORBILIDADES.keys():
    columnas.append(f"Comorbilidad_{comorbilidad}")

# Añadir columnas para efectos adversos, gravedad y frecuencia
for farmaco in FARMACO_DISTRIBUCION.keys():
    if farmaco in ef_adversos_dict:
        for efecto in ef_adversos_dict[farmaco]:
            nombre_efecto = efecto["Efecto_Adverso"].replace(" ", "_")
            columnas.extend([f"EA_{nombre_efecto}", f"Gravedad_{nombre_efecto}", f"Frecuencia_{nombre_efecto}"])

csv_writer.writerow(columnas)

# Generar datos
for i in range(1, NUM_PACIENTES + 1):
    genero = np.random.choice(list(GENERO_DISTRIBUCION.keys()), p=list(GENERO_DISTRIBUCION.values()))

    # Generar IMC categórico
    if genero == "Femenino":
        probabilidades = [0.039, 0.502, 0.271, 0.126]
        probabilidades = [p / sum(probabilidades) for p in probabilidades]
        imc_categorica = np.random.choice([1, 2, 3, 4], p=probabilidades)
    else:
        probabilidades = [0.011, 0.38, 0.405, 0.152]
        probabilidades = [p / sum(probabilidades) for p in probabilidades]
        imc_categorica = np.random.choice([1, 2, 3, 4], p=probabilidades)

    # Generar edad basada en la categoría de IMC y el género
    edad = generar_edad(imc_categorica, genero)

    # Asignar IMC según la categoría
    if imc_categorica == 1:
        imc = np.random.uniform(16, 18.5)
    elif imc_categorica == 2:
        imc = np.random.uniform(18.5, 25)
    elif imc_categorica == 3:
        imc = np.random.uniform(25, 30)
    else:
        imc = np.random.uniform(30, 40)

    # Generar altura y peso basados en el IMC y siguiendo una distribución normal por género
    if genero == "Femenino":
        altura = np.random.normal(1.65, 0.06)
        peso = np.random.normal(imc * (altura ** 2), 5)
    else:
        altura = np.random.normal(1.75, 0.07)
        peso = np.random.normal(imc * (altura ** 2), 6)

    altura = max(1.5, min(altura, 1.9))
    peso = max(40, min(peso, 150))

    # Asignar comorbilidades
    comorbilidades = {f"Comorbilidad_{comorbilidad}": 0 for comorbilidad in COMORBILIDADES}
    if random.random() < COMORBIDITY_RATE:
        for comorbilidad, probabilidad in COMORBILIDADES.items():
            if random.random() < probabilidad:
                comorbilidades[f"Comorbilidad_{comorbilidad}"] = 1

    # Asignar fármaco considerando restricciones
    farmacos_posibles = list(FARMACO_DISTRIBUCION.keys())
    probabilidades = list(FARMACO_DISTRIBUCION.values())
    farmaco = None

    while farmaco is None:
        # Normalizar las probabilidades
        probabilidades_normalizadas = [p / sum(probabilidades) for p in probabilidades]

        # Seleccionar un fármaco candidato
        farmaco_candidato = np.random.choice(farmacos_posibles, p=probabilidades_normalizadas)

        # Verificar restricciones
        if verificar_restricciones(farmaco_candidato, genero, edad, comorbilidades, imc_categorica):
            farmaco = farmaco_candidato
        else:
            # Eliminar el fármaco de la lista de posibles y ajustar las probabilidades
            indice = farmacos_posibles.index(farmaco_candidato)
            farmacos_posibles.pop(indice)
            probabilidades.pop(indice)

            # Si no hay fármacos posibles, asignar el fármaco con menos restricciones
            if not farmacos_posibles:
                farmaco = min(RESTRICCIONES_FARMACOS.keys(), key=lambda x: len(RESTRICCIONES_FARMACOS[x]))

    # Asignar efectos adversos
    efectos_adversos = {}
    if random.random() < ADVERSE_EFFECTS_RATE and farmaco in ef_adversos_dict:
        efectos_seleccionados = random.sample(ef_adversos_dict[farmaco], min(4, len(ef_adversos_dict[farmaco])))
        for efecto in efectos_seleccionados:
            nombre_efecto = efecto["Efecto_Adverso"].replace(" ", "_")
            efectos_adversos[f"EA_{nombre_efecto}"] = 1
            efectos_adversos[f"Gravedad_{nombre_efecto}"] = efecto["Gravedad"]
            efectos_adversos[f"Frecuencia_{nombre_efecto}"] = efecto["Frecuencia"]

    # Crear fila con todos los datos
    fila = [
        i, edad, peso, round(altura * 100), imc, imc_categorica, genero, farmaco,
        ADHERENCIA[farmaco](), PRECIO[farmaco], EFICACIA[farmaco]()
    ]

    # Añadir comorbilidades
    fila.extend(comorbilidades.values())

    # Añadir efectos adversos
    for col in columnas[len(fila):]:
        fila.append(efectos_adversos.get(col, 0))

    csv_writer.writerow(fila)

# Guardar los datos generados en un archivo CSV
output.seek(0)  # Reiniciar el puntero al inicio del StringIO
df_generado = pd.read_csv(output)
df_generado.to_csv("test.csv", index=False)

print("Datos generados guardados en 'test.csv'")

Datos generados guardados en 'test.csv'


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel("test.xlsx")
df

,id,edad,peso,altura,IMC,IMC_categorica,genero,farmaco,Adherencia,Precio,...,Frecuencia_VÃ©rtigo.1,EA_EstreÃ±imiento.4,Gravedad_EstreÃ±imiento.4,Frecuencia_EstreÃ±imiento.4,EA_Prurito.3,Gravedad_Prurito.3,Frecuencia_Prurito.3,EA_Dolor_en_el_sitio_de_inyecciÃ³n.1,Gravedad_Dolor_en_el_sitio_de_inyecciÃ³n.1,Frecuencia_Dolor_en_el_sitio_de_inyecciÃ³n.1
0,1,27,89.098910,156,38.972365,4,Femenino,Amitriptilina,0.493585,16.96,...,0,0,0,0,0,0,0,0,0,0
1,2,60,55.936648,158,23.832524,2,Femenino,Amitriptilina,0.534620,16.96,...,0,0,0,0,0,0,0,0,0,0
2,3,22,51.462954,160,22.337769,2,Femenino,Galcanezumab,1.000000,5400.00,...,0,0,0,0,0,0,0,0,0,0
3,4,34,53.060743,168,21.065284,2,Femenino,Fremanezumab,1.000000,5976.00,...,0,0,0,0,0,0,0,0,0,0
4,5,27,63.069577,154,28.510922,3,Femenino,Amitriptilina,0.556055,16.96,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,22,55.782883,169,21.149569,2,Femenino,Amitriptilina,0.593050,16.96,...,0,0,0,0,0,0,0,0,0,0
1996,1997,64,48.665475,173,19.067623,2,Femenino,Amitriptilina,0.570101,16.96,...,0,0,0,0,0,0,0,0,0,0
1997,1998,28,97.193609,173,31.192866,4,Femenino,Amitriptilina,0.572196,16.96,...,0,0,0,0,0,0,0,0,0,0
1998,1999,52,71.475071,170,24.893372,2,Femenino,Erenumab,1.000000,5340.00,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

# Eliminar columnas que terminan en .1, .2, .3 y .4
columnas_a_eliminar = [col for col in df.columns if col.endswith('.1') or col.endswith('.2') or col.endswith('.3') or col.endswith('.4')]
df = df.drop(columns=columnas_a_eliminar)

# Guardar el archivo modificado
df.to_excel('test_modificado.xlsx', index=False)

In [ ]:
df

,id,edad,peso,altura,IMC,IMC_categorica,genero,farmaco,Adherencia,Precio,...,Frecuencia_InduraciÃ³n,EA_Eritema,Gravedad_Eritema,Frecuencia_Eritema,EA_ReacciÃ³n_anafilÃ¡ctica,Gravedad_ReacciÃ³n_anafilÃ¡ctica,Frecuencia_ReacciÃ³n_anafilÃ¡ctica,EA_Anafilaxis,Gravedad_Anafilaxis,Frecuencia_Anafilaxis
0,1,27,89.098910,156,38.972365,4,Femenino,Amitriptilina,0.493585,16.96,...,0,0,0,0,0,0,0,0,0,0
1,2,60,55.936648,158,23.832524,2,Femenino,Amitriptilina,0.534620,16.96,...,0,0,0,0,0,0,0,0,0,0
2,3,22,51.462954,160,22.337769,2,Femenino,Galcanezumab,1.000000,5400.00,...,0,0,0,0,0,0,0,0,0,0
3,4,34,53.060743,168,21.065284,2,Femenino,Fremanezumab,1.000000,5976.00,...,0,0,0,0,0,0,0,0,0,0
4,5,27,63.069577,154,28.510922,3,Femenino,Amitriptilina,0.556055,16.96,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,22,55.782883,169,21.149569,2,Femenino,Amitriptilina,0.593050,16.96,...,0,0,0,0,0,0,0,0,0,0
1996,1997,64,48.665475,173,19.067623,2,Femenino,Amitriptilina,0.570101,16.96,...,0,0,0,0,0,0,0,0,0,0
1997,1998,28,97.193609,173,31.192866,4,Femenino,Amitriptilina,0.572196,16.96,...,0,0,0,0,0,0,0,0,0,0
1998,1999,52,71.475071,170,24.893372,2,Femenino,Erenumab,1.000000,5340.00,...,0,0,0,0,0,0,0,0,0,0
